In [ ]:
# Estados de la conversación
(ACTION_TYPE, MOVEMENT_TYPE, AMOUNT, CURRENCY, DESCRIPTION, PAYMENT_METHOD, 
 COMMENT, CONTINUE, UPDATE_ACCOUNT, CONSULT_ACCOUNT, UPDATE_AMOUNT) = range(11)

reply_keyboard = [['➕ Agregar Movimiento', '💰 Actualizar Saldos', '🔍 Consultar Saldos']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text(
        "Bienvenid@ a *Capybara*. ¿Qué deseas hacer?",
        parse_mode='Markdown',
        reply_markup=markup
    )
    return ACTION_TYPE

async def action_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_choice = update.message.text
    if user_choice == '💰 Actualizar Saldos':
        context.user_data['action_type'] = 'Actualizar Saldos'
        await update.message.reply_text(
            "Selecciona la cuenta que deseas actualizar:",
            reply_markup=ReplyKeyboardMarkup([
                ['Efectivo Pesos', 'Efectivo Dólares'],
                ['Naranja', 'Mercado Pago'],
                ['CA $ Galicia', 'CA USD Galicia'],
                ['Cocos', 'Binance'],
                ['Bull Market', 'Lemon']
            ], one_time_keyboard=True)
        )
        return UPDATE_ACCOUNT

    elif user_choice == '🔍 Consultar Saldos':
        context.user_data['action_type'] = 'Consultar Saldos'
        await update.message.reply_text(
            "Selecciona la cuenta que deseas consultar:",
            reply_markup=ReplyKeyboardMarkup([
                ['Efectivo Pesos', 'Efectivo Dólares'],
                ['Naranja', 'Mercado Pago'],
                ['CA $ Galicia', 'CA USD Galicia'],
                ['Cocos', 'Binance'],
                ['Bull Market', 'Lemon']
            ], one_time_keyboard=True)
        )
        return CONSULT_ACCOUNT

# Flujos separados para consultar y actualizar saldos
async def update_account(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['account'] = update.message.text
    await update.message.reply_text(
        f"Ingrese el nuevo saldo para la cuenta {update.message.text}:"
    )
    return UPDATE_AMOUNT

async def consult_account(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    account = update.message.text
    saldo = obtener_saldo(account)
    await update.message.reply_text(f"El saldo de la cuenta {account} es {saldo}.")
    await update.message.reply_text(
        "¿Quieres realizar otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

async def update_amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    account = context.user_data['account']
    new_amount = update.message.text
    actualizar_saldo(account, new_amount)
    await update.message.reply_text(
        f"El saldo de {account} ha sido actualizado a {new_amount}."
    )
    await update.message.reply_text(
        "¿Quieres realizar otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

# Manejo de saldos en archivo CSV
def obtener_saldo(cuenta):
    try:
        df_saldos = pd.read_csv("saldos.csv")
        if cuenta in df_saldos["cuenta"].values:
            return df_saldos.loc[df_saldos["cuenta"] == cuenta, "saldo"].values[0]
        return "Cuenta no encontrada"
    except FileNotFoundError:
        return "Archivo de saldos no encontrado"

def actualizar_saldo(cuenta, monto):
    try:
        df_saldos = pd.read_csv("saldos.csv")
    except FileNotFoundError:
        df_saldos = pd.DataFrame(columns=["cuenta", "saldo", "fecha_actualizacion"])
    
    fecha_actualizacion = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    if cuenta in df_saldos["cuenta"].values:
        df_saldos.loc[df_saldos["cuenta"] == cuenta, ["saldo", "fecha_actualizacion"]] = [monto, fecha_actualizacion]
    else:
        new_row = pd.DataFrame([[cuenta, monto, fecha_actualizacion]], columns=["cuenta", "saldo", "fecha_actualizacion"])
        df_saldos = pd.concat([df_saldos, new_row], ignore_index=True)
    
    df_saldos.to_csv("saldos.csv", index=False)

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Operación cancelada. ¡Hasta luego!")
    return ConversationHandler.END

def main():
    application = Application.builder().token(TOKEN).build()
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            ACTION_TYPE: [MessageHandler(filters.Regex('^(➕ Agregar Movimiento|💰 Actualizar Saldos|🔍 Consultar Saldos)$'), action_type)],
            UPDATE_ACCOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_account)],
            CONSULT_ACCOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, consult_account)],
            UPDATE_AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_amount)],
            CONTINUE: [MessageHandler(filters.Regex('^(Sí|No)$'), start)],
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )
    application.add_handler(conv_handler)
    application.run_polling()

if __name__ == '__main__':
    main()

In [15]:
!pip install python-telegram-bot
!pip install nest_asyncio

  Using cached python_telegram_bot-21.2-py3-none-any.whl (629 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)


In [16]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, CallbackContext, ApplicationBuilder, CommandHandler, ContextTypes
import pandas as pd
import logging
import nest_asyncio
import asyncio

In [8]:
TOKEN = '7465050668:AAExMbHOI9PEsC2kzeFwVfdtNcjM_HBJ-jE'

Configuracion de Excepciones

In [12]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

In [13]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

In [18]:
if __name__ == '__main__':
    # Aplicar nest_asyncio para permitir múltiples bucles de eventos
    nest_asyncio.apply()
    
    application = ApplicationBuilder().token(TOKEN).build()
    
    start_handler = CommandHandler('start', start)
    application.add_handler(start_handler)
    
    # Iniciar el bucle de eventos dentro del entorno de notebook
    loop = asyncio.get_event_loop()
    loop.run_until_complete(application.initialize())
    loop.run_until_complete(application.start())
    loop.run_forever()

2024-06-01 17:53:44,773 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7465050668:AAExMbHOI9PEsC2kzeFwVfdtNcjM_HBJ-jE/getMe "HTTP/1.1 200 OK"
2024-06-01 17:53:44,776 - telegram.ext.Application - INFO - Application started


KeyboardInterrupt: 

: 

In [ ]:
import logging
from telegram import Update, ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Application, CommandHandler, MessageHandler, ConversationHandler, ContextTypes, filters, CallbackQueryHandler
import pandas as pd

TOKEN = '7465050668:AAExMbHOI9PEsC2kzeFwVfdtNcjM_HBJ-jE'

# Configuración de alertas
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Variables
MOVEMENT_TYPE, AMOUNT, CURRENCY, DESCRIPTION, PAYMENT_METHOD, COMMENT, CONTINUE = range(7)

# Opciones
reply_keyboard = [['🤑 Ingreso', '🚨 Gasto']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

# Inicio
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Mensaje de bienvenida y elección del tipo de movimiento
    await update.message.reply_text(
        "Bienvenid@ a *Capybara*, tu bot para controlar tus finanzas. "
        "Elegí el *tipo de movimiento* que quieras registrar.",
        parse_mode='Markdown',
        reply_markup=markup
    )
    # Devuelvo variable tipo de movimiento
    return MOVEMENT_TYPE

# Monto
async def movement_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo tipo de movimiento en la base
    user = update.message.from_user
    context.user_data['movement_type'] = update.message.text
    logging.info("Tipo de movimiento de %s: %s", user.first_name, update.message.text)
    
    # Pregunto por Monto
    await update.message.reply_text(
        "Perfecto. Ahora completa la siguiente información:\n 💰 *Monto*: \n **Ejemplo: 1000**",
        parse_mode='Markdown'
    )
    # Devuelvo variable Monto
    return AMOUNT


# Moneda
async def amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Monto en la base
    user = update.message.from_user
    context.user_data['amount'] = update.message.text
    logging.info("Monto de %s: %s", user.first_name, update.message.text)
    
    # Pregunto por Moneda
    await update.message.reply_text(
        " *Moneda*:",
        parse_mode='Markdown',
        reply_markup=ReplyKeyboardMarkup([['💲 Peso', '💵  Dólar', '👾 Cripto']], one_time_keyboard=True)
    )
    
    # Devuelvo variable Moneda
    return CURRENCY

# Descripcion
async def currency(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Moneda en la base
    user = update.message.from_user
    context.user_data['currency'] = update.message.text
    logging.info("Moneda de %s: %s", user.first_name, update.message.text)
    
    # Opciones para descripción según el tipo de movimiento
    if context.user_data['movement_type'] == '🤑 Ingreso':
        description_buttons = [
            [InlineKeyboardButton("Sueldo", callback_data='Sueldo')],
            [InlineKeyboardButton("Inversión", callback_data='Inversión')],
            [InlineKeyboardButton("Regalo", callback_data='Ingreso_Regalo')]
        ]
    else:
        description_buttons = [
            [InlineKeyboardButton("Salud", callback_data='Salud')],
            [InlineKeyboardButton("Impuesto", callback_data='Impuesto')],
            [InlineKeyboardButton("Recital", callback_data='Recital')],
            [InlineKeyboardButton("Salida", callback_data='Salida')],
            [InlineKeyboardButton("Hogar", callback_data='Hogar')],
            [InlineKeyboardButton("Supermercado", callback_data='Supermercado')],
            [InlineKeyboardButton("Transporte", callback_data='Transporte')],
            [InlineKeyboardButton("Mascota", callback_data='Mascota')],
            [InlineKeyboardButton("Donación", callback_data='Donación')],
            [InlineKeyboardButton("Auto", callback_data='Auto')],
            [InlineKeyboardButton("Estudio", callback_data='Estudio')],
            [InlineKeyboardButton("Gimnasio", callback_data='Gimnasio')],
            [InlineKeyboardButton("Regalo", callback_data='Gasto_Regalo')],
            [InlineKeyboardButton("Banco/Resumen", callback_data='Banco/Resumen')],
            [InlineKeyboardButton("Viaje", callback_data='Viaje')]
        ]
    
    # Pregunto por Descripcion
    await update.message.reply_text(
        "Descripción:",
        reply_markup=InlineKeyboardMarkup(description_buttons)
    )
    return DESCRIPTION

# Función para manejar la descripción
async def description(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Descripcion en la base
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['description'] = query.data
    logging.info("Descripción de %s: %s", user.first_name, query.data)
    
    payment_buttons = [
        [InlineKeyboardButton("Efectivo Pesos", callback_data='Efectivo Pesos')],
        [InlineKeyboardButton("Efectivo Dólares", callback_data='Efectivo Dólares')],
        [InlineKeyboardButton("Débito Naranja", callback_data='Débito Naranja')],
        [InlineKeyboardButton("Crédito Naranja", callback_data='Crédito Naranja')],
        [InlineKeyboardButton("Crédito Visa Naranja", callback_data='Crédito Visa Naranja')],
        [InlineKeyboardButton("Crédito Visa Galicia", callback_data='Crédito Visa Galicia')],
        [InlineKeyboardButton("Crédito Master Galicia", callback_data='Crédito Master Galicia')],
        [InlineKeyboardButton("Débito Macro", callback_data='Débito Macro')],
        [InlineKeyboardButton("Crédito Visa Macro", callback_data='Crédito Visa Macro')],
        [InlineKeyboardButton("Crédito Master Macro", callback_data='Crédito Master Macro')],
    ]
    
    await query.message.reply_text(
        "¿De qué forma se realizó el movimiento?",
        reply_markup=InlineKeyboardMarkup(payment_buttons)
    )
    return PAYMENT_METHOD

# Comentario
async def payment_method(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['payment_method'] = query.data
    logging.info("Forma de pago de %s: %s", user.first_name, query.data)

    await query.message.reply_text(
        "¿Desea agregar un comentario?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return COMMENT

# Función para manejar el comentario o continuar si es 'No'
async def comment(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Verifico si el usuario quiere agregar un comentario
    if update.message.text == 'Sí':
        context.user_data['awaiting_comment'] = True  # Indicamos que estamos esperando un comentario
        await update.message.reply_text("Comentario:")
        return CONTINUE
    else:
        # Si el usuario no quiere agregar un comentario, guardo un valor vacío
        context.user_data['comment'] = ""  # Comentario vacío si el usuario dijo "No"

        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # No estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿desea registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

# Función para manejar la respuesta del comentario
async def handle_comment_text(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Solo se ejecuta si estábamos esperando un comentario
    if context.user_data.get('awaiting_comment', False):
        # Guardo el comentario en la base de datos
        user = update.message.from_user
        context.user_data['comment'] = update.message.text
        logging.info("Comentario de %s: %s", user.first_name, update.message.text)
        
        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # Ya no estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿desea registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

    # Si no se esperaba un comentario, manejamos la respuesta de continuar
    return await continue_conversation(update, context)

# Función para manejar la continuación
async def continue_conversation(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Comienzo de nuevo si la respuesta es 'Sí'
    if update.message.text == 'Sí':
        return await start(update, context)
    
    # Termino la conversación si la respuesta es 'No'
    else:
        await update.message.reply_text("Gracias por usar TuCapybaraBot. ¡Hasta luego!")
        return ConversationHandler.END

# Función para guardar datos en un CSV
def guardar_datos(data):
    try:
        df = pd.read_csv("finanzas.csv")
    except FileNotFoundError:
        df = pd.DataFrame(columns=["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"])

    # Crear un DataFrame temporal con los nuevos datos
    new_data = pd.DataFrame([data])

    # Asignar número de movimiento (número de fila)
    new_data["numero_movimiento"] = len(df) + 1
    
    # Asignar fecha actual
    new_data["fecha"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    # Reordenar las columnas para tener "numero_movimiento" al principio y "fecha" al final
    new_data = new_data[["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"]]

    # Concatenar el DataFrame temporal con el DataFrame existente
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("finanzas.csv", index=False)

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text('Operación cancelada. ¡Hasta luego!')
    return ConversationHandler.END

def main() -> None:
    application = Application.builder().token(TOKEN).build()
    
    conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        MOVEMENT_TYPE: [MessageHandler(filters.Regex('^(🤑 Ingreso|🚨 Gasto)$'), movement_type)],
        AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, amount)],
        CURRENCY: [MessageHandler(filters.Regex('^(💲 Peso|💵  Dólar|👾 Cripto)$'), currency)],
        DESCRIPTION: [CallbackQueryHandler(description)],
        PAYMENT_METHOD: [CallbackQueryHandler(payment_method)],
        COMMENT: [MessageHandler(filters.Regex('^(Sí|No)$'), comment)],
        CONTINUE: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_comment_text),
                   MessageHandler(filters.Regex('^(Sí|No)$'), continue_conversation)],
    },
    fallbacks=[CommandHandler('cancel', cancel)],
)

    application.add_handler(conv_handler)

    application.run_polling()

if __name__ == '__main__':
    main()

In [ ]:
import logging
from telegram import Update, ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Application, CommandHandler, MessageHandler, ConversationHandler, ContextTypes, filters, CallbackQueryHandler
import pandas as pd

TOKEN = '7465050668:AAExMbHOI9PEsC2kzeFwVfdtNcjM_HBJ-jE'

# Configuración de alertas
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Variables
ACTION_TYPE, MOVEMENT_TYPE, AMOUNT, CURRENCY, DESCRIPTION, PAYMENT_METHOD, COMMENT, CONTINUE, ACCOUNT, UPDATE_AMOUNT = range(10)

# Opciones iniciales
reply_keyboard = [['➕ Agregar Movimiento', '💰 Actualizar Saldos']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

# Inicializar el archivo de saldos si no existe
def inicializar_saldos():
    cuentas = ["Efectivo Pesos", "Efectivo Dólares", "Débito Naranja", "Crédito Naranja", 
               "Crédito Visa Naranja", "Crédito Visa Galicia", "Crédito Master Galicia", 
               "Débito Macro", "Crédito Visa Macro", "Crédito Master Macro"]

    try:
        df = pd.read_csv("saldos.csv")
    except FileNotFoundError:
        df = pd.DataFrame({"Cuenta": cuentas, "Saldo": [0] * len(cuentas)})
        df.to_csv("saldos.csv", index=False)

# Función para actualizar el saldo de una cuenta específica
def actualizar_saldo(cuenta, nuevo_saldo):
    try:
        df = pd.read_csv("saldos.csv")
        df.loc[df['Cuenta'] == cuenta, 'Saldo'] = nuevo_saldo
        df.to_csv("saldos.csv", index=False)
        print(f"El saldo de {cuenta} ha sido actualizado a {nuevo_saldo}.")
    except FileNotFoundError:
        print("El archivo de saldos no existe. Inicializando archivo...")
        inicializar_saldos()
        actualizar_saldo(cuenta, nuevo_saldo)

# Inicio
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Mensaje de bienvenida y elección de acción
    await update.message.reply_text(
        "Bienvenid@ a *Capybara*, tu bot para controlar tus finanzas. "
        "¿Qué deseas hacer?",
        parse_mode='Markdown',
        reply_markup=markup
    )
    return ACTION_TYPE

# Elección entre agregar movimiento o actualizar saldos
async def action_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_choice = update.message.text
    
    if user_choice == '➕ Agregar Movimiento':
        # Proseguir con el flujo de agregar movimiento
        context.user_data['action_type'] = 'Agregar Movimiento'
        await update.message.reply_text(
            "Elegí el *tipo de movimiento* que quieras registrar.",
            parse_mode='Markdown',
            reply_markup=ReplyKeyboardMarkup([['🤑 Ingreso', '🚨 Gasto']], one_time_keyboard=True)
        )
        return MOVEMENT_TYPE

    elif user_choice == '💰 Actualizar Saldos':
        # Preguntar por la cuenta a actualizar
        context.user_data['action_type'] = 'Actualizar Saldos'
        await update.message.reply_text(
            "Elija su cuenta:",
            reply_markup=ReplyKeyboardMarkup([
                ['Efectivo Pesos', 'Efectivo Dólares'],
                ['Débito Naranja', 'Crédito Naranja'],
                ['Crédito Visa Naranja', 'Crédito Visa Galicia'],
                ['Crédito Master Galicia', 'Débito Macro'],
                ['Crédito Visa Macro', 'Crédito Master Macro']
            ], one_time_keyboard=True)
        )
        return ACCOUNT

# Monto
async def movement_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    context.user_data['movement_type'] = update.message.text
    logging.info("Tipo de movimiento de %s: %s", user.first_name, update.message.text)
    
    await update.message.reply_text(
        "Perfecto. Ahora completa la siguiente información:\n 💰 *Monto*: \n **Ejemplo: 1000**",
        parse_mode='Markdown'
    )
    return AMOUNT

# Moneda
async def amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Monto en la base
    user = update.message.from_user
    context.user_data['amount'] = update.message.text
    logging.info("Monto de %s: %s", user.first_name, update.message.text)
    
    # Pregunto por Moneda
    await update.message.reply_text(
        " *Moneda*:",
        parse_mode='Markdown',
        reply_markup=ReplyKeyboardMarkup([['💲 Peso', '💵  Dólar', '👾 Cripto']], one_time_keyboard=True)
    )
    
    # Devuelvo variable Moneda
    return CURRENCY

# Descripcion
async def currency(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Moneda en la base
    user = update.message.from_user
    context.user_data['currency'] = update.message.text
    logging.info("Moneda de %s: %s", user.first_name, update.message.text)
    
    # Opciones para descripción según el tipo de movimiento
    if context.user_data['movement_type'] == '🤑 Ingreso':
        description_buttons = [
            [InlineKeyboardButton("Sueldo", callback_data='Sueldo')],
            [InlineKeyboardButton("Inversión", callback_data='Inversión')],
            [InlineKeyboardButton("Regalo", callback_data='Ingreso_Regalo')]
        ]
    else:
        description_buttons = [
            [InlineKeyboardButton("Salud", callback_data='Salud')],
            [InlineKeyboardButton("Impuesto", callback_data='Impuesto')],
            [InlineKeyboardButton("Recital", callback_data='Recital')],
            [InlineKeyboardButton("Salida", callback_data='Salida')],
            [InlineKeyboardButton("Hogar", callback_data='Hogar')],
            [InlineKeyboardButton("Supermercado", callback_data='Supermercado')],
            [InlineKeyboardButton("Transporte", callback_data='Transporte')],
            [InlineKeyboardButton("Mascota", callback_data='Mascota')],
            [InlineKeyboardButton("Donación", callback_data='Donación')],
            [InlineKeyboardButton("Auto", callback_data='Auto')],
            [InlineKeyboardButton("Estudio", callback_data='Estudio')],
            [InlineKeyboardButton("Gimnasio", callback_data='Gimnasio')],
            [InlineKeyboardButton("Regalo", callback_data='Gasto_Regalo')],
            [InlineKeyboardButton("Banco/Resumen", callback_data='Banco/Resumen')],
            [InlineKeyboardButton("Viaje", callback_data='Viaje')]
        ]
    
    # Pregunto por Descripcion
    await update.message.reply_text(
        "Descripción:",
        reply_markup=InlineKeyboardMarkup(description_buttons)
    )
    return DESCRIPTION

# Función para manejar la descripción
async def description(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Descripcion en la base
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['description'] = query.data
    logging.info("Descripción de %s: %s", user.first_name, query.data)
    
    payment_buttons = [
        [InlineKeyboardButton("Efectivo Pesos", callback_data='Efectivo Pesos')],
        [InlineKeyboardButton("Efectivo Dólares", callback_data='Efectivo Dólares')],
        [InlineKeyboardButton("Débito Naranja", callback_data='Débito Naranja')],
        [InlineKeyboardButton("Crédito Naranja", callback_data='Crédito Naranja')],
        [InlineKeyboardButton("Crédito Visa Naranja", callback_data='Crédito Visa Naranja')],
        [InlineKeyboardButton("Crédito Visa Galicia", callback_data='Crédito Visa Galicia')],
        [InlineKeyboardButton("Crédito Master Galicia", callback_data='Crédito Master Galicia')],
        [InlineKeyboardButton("Débito Macro", callback_data='Débito Macro')],
        [InlineKeyboardButton("Crédito Visa Macro", callback_data='Crédito Visa Macro')],
        [InlineKeyboardButton("Crédito Master Macro", callback_data='Crédito Master Macro')],
    ]
    
    await query.message.reply_text(
        "¿De qué forma se realizó el movimiento?",
        reply_markup=InlineKeyboardMarkup(payment_buttons)
    )
    return PAYMENT_METHOD

# Comentario
async def payment_method(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['payment_method'] = query.data
    logging.info("Forma de pago de %s: %s", user.first_name, query.data)

    await query.message.reply_text(
        "¿Desea agregar un comentario?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return COMMENT

# Función para manejar el comentario o continuar si es 'No'
async def comment(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Verifico si el usuario quiere agregar un comentario
    if update.message.text == 'Sí':
        context.user_data['awaiting_comment'] = True  # Indicamos que estamos esperando un comentario
        await update.message.reply_text("Comentario:")
        return CONTINUE
    else:
        # Si el usuario no quiere agregar un comentario, guardo un valor vacío
        context.user_data['comment'] = ""  # Comentario vacío si el usuario dijo "No"

        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # No estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿desea registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

# Función para manejar la respuesta del comentario
async def handle_comment_text(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Solo se ejecuta si estábamos esperando un comentario
    if context.user_data.get('awaiting_comment', False):
        # Guardo el comentario en la base de datos
        user = update.message.from_user
        context.user_data['comment'] = update.message.text
        logging.info("Comentario de %s: %s", user.first_name, update.message.text)
        
        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # Ya no estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿desea registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

    # Si no se esperaba un comentario, manejamos la respuesta de continuar
    return await continue_conversation(update, context)

# Función para manejar la cuenta elegida
async def account(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    context.user_data['account'] = update.message.text
    logging.info("Cuenta seleccionada por %s: %s", user.first_name, update.message.text)

    await update.message.reply_text(
        f"Actualice con el nuevo monto para {update.message.text}:"
    )
    return UPDATE_AMOUNT

# Función para actualizar el saldo con el nuevo monto
async def update_amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    context.user_data['new_amount'] = update.message.text
    logging.info("Nuevo saldo para %s: %s", context.user_data['account'], update.message.text)

    await update.message.reply_text(
        f"El saldo de {context.user_data['account']} ha sido actualizado a {update.message.text}."
    )

    await update.message.reply_text(
        "¿Desea hacer otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

# Función para manejar la continuación
async def continue_conversation(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Comienzo de nuevo si la respuesta es 'Sí'
    if update.message.text == 'Sí':
        return await start(update, context)
    
    # Termino la conversación si la respuesta es 'No'
    else:
        await update.message.reply_text("Gracias por usar TuCapybaraBot. ¡Hasta luego!")
        return ConversationHandler.END

# Función para guardar datos en un CSV
def guardar_datos(data):
    try:
        df = pd.read_csv("finanzas.csv")
    except FileNotFoundError:
        df = pd.DataFrame(columns=["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"])

    # Crear un DataFrame temporal con los nuevos datos
    new_data = pd.DataFrame([data])

    # Asignar número de movimiento (número de fila)
    new_data["numero_movimiento"] = len(df) + 1
    
    # Asignar fecha actual
    new_data["fecha"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    # Reordenar las columnas para tener "numero_movimiento" al principio y "fecha" al final
    new_data = new_data[["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"]]

    # Concatenar el DataFrame temporal con el DataFrame existente
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("finanzas.csv", index=False)
    

# Asegurarse de inicializar el archivo de saldos al principio
inicializar_saldos()

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text('Operación cancelada. ¡Hasta luego!')
    return ConversationHandler.END

def main() -> None:
    application = Application.builder().token(TOKEN).build()
    
    conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
       ACTION_TYPE: [MessageHandler(filters.Regex('^(➕ Agregar Movimiento|💰 Actualizar Saldos)$'), action_type)],
       MOVEMENT_TYPE: [MessageHandler(filters.Regex('^(🤑 Ingreso|🚨 Gasto)$'), movement_type)],
            ACCOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, account)],
            UPDATE_AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_amount)],
            AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, amount)],
            CURRENCY: [MessageHandler(filters.TEXT & ~filters.COMMAND, currency)],
            COMMENT: [MessageHandler(filters.Regex('^(Sí|No)$'), comment)],
            CONTINUE: [MessageHandler(filters.Regex('^(Sí|No)$'), continue_conversation)],
            DESCRIPTION: [CallbackQueryHandler(description)],
            PAYMENT_METHOD: [CallbackQueryHandler(payment_method)],
            CONTINUE: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_comment_text)],
    },
    fallbacks=[CommandHandler('cancel', cancel)],
)

    application.add_handler(conv_handler)

    application.run_polling()

if __name__ == '__main__':
    main()

In [ ]:
import logging
from telegram import Update, ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Application, CommandHandler, MessageHandler, ConversationHandler, ContextTypes, filters, CallbackQueryHandler
import pandas as pd

TOKEN = '7465050668:AAExMbHOI9PEsC2kzeFwVfdtNcjM_HBJ-jE'

# Configuración de alertas
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Variables
(ACTION_TYPE, MOVEMENT_TYPE, AMOUNT, CURRENCY, DESCRIPTION, PAYMENT_METHOD, COMMENT, CONTINUE, UPDATE_ACCOUNT, CONSULT_ACCOUNT, UPDATE_AMOUNT) = range(11)

# Opciones iniciales
reply_keyboard = [['➕ Agregar Movimiento', '💰 Actualizar Saldos', '🔍 Consultar Saldos']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

# Inicio
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Mensaje de bienvenida y elección de acción
    await update.message.reply_text(
        "Bienvenid@ a *Capybara*, tu bot para controlar tus finanzas. "
        "¿Qué deseás hacer?",
        parse_mode='Markdown',
        reply_markup=markup
    )
    return ACTION_TYPE

# Elección entre agregar movimiento, actualizar o consultar saldos
async def action_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user_choice = update.message.text
    
    if user_choice == '➕ Agregar Movimiento':
        context.user_data['action_type'] = 'Agregar Movimiento'
        await update.message.reply_text(
            "Elegí el *tipo de movimiento* que quieras registrar.",
            parse_mode='Markdown',
            reply_markup=ReplyKeyboardMarkup([['🤑 Ingreso', '🚨 Gasto']], one_time_keyboard=True)
        )
        return MOVEMENT_TYPE

    elif user_choice == '💰 Actualizar Saldos':
        context.user_data['action_type'] = 'Actualizar Saldos'
        await update.message.reply_text(
            "Selecciona la cuenta que deseas actualizar:",
            reply_markup=ReplyKeyboardMarkup([
                ['Efectivo Pesos', 'Efectivo Dólares'],
                ['Naranja', 'Mercado Pago'],
                ['CA $ Galicia', 'CA USD Galicia'],
                ['Cocos', 'Binance'],
                ['Bull Market', 'Lemon']
            ], one_time_keyboard=True)
        )
        return UPDATE_ACCOUNT

    elif user_choice == '🔍 Consultar Saldos':
        context.user_data['action_type'] = 'Consultar Saldos'
        await update.message.reply_text(
            "Selecciona la cuenta que deseas consultar:",
            reply_markup=ReplyKeyboardMarkup([
                ['Efectivo Pesos', 'Efectivo Dólares'],
                ['Naranja', 'Mercado Pago'],
                ['CA $ Galicia', 'CA USD Galicia'],
                ['Cocos', 'Binance'],
                ['Bull Market', 'Lemon']
            ], one_time_keyboard=True)
        )
        return CONSULT_ACCOUNT

# Monto
async def movement_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    context.user_data['movement_type'] = update.message.text
    logging.info("Tipo de movimiento de %s: %s", user.first_name, update.message.text)
    
    await update.message.reply_text(
        "Perfecto. Ahora completá la siguiente información:\n 💰 *Monto*: \n **Ejemplo: 1000**",
        parse_mode='Markdown'
    )
    return AMOUNT

# Moneda
async def amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Monto en la base
    user = update.message.from_user
    context.user_data['amount'] = update.message.text
    logging.info("Monto de %s: %s", user.first_name, update.message.text)
    
    # Pregunto por Moneda
    await update.message.reply_text(
        "Elegí la *moneda*:",
        parse_mode='Markdown',
        reply_markup=ReplyKeyboardMarkup([['💲 Peso', '💵  Dólar', '👾 Cripto']], one_time_keyboard=True)
    )
    
    # Devuelvo variable Moneda
    return CURRENCY

# Descripcion
async def currency(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Moneda en la base
    user = update.message.from_user
    context.user_data['currency'] = update.message.text
    logging.info("Moneda de %s: %s", user.first_name, update.message.text)
    
    # Opciones para descripción según el tipo de movimiento
    if context.user_data['movement_type'] == '🤑 Ingreso':
        description_buttons = [
            [InlineKeyboardButton("Sueldo", callback_data='Sueldo')],
            [InlineKeyboardButton("Inversión", callback_data='Inversión')],
            [InlineKeyboardButton("Devolución", callback_data='Devolución')],
            [InlineKeyboardButton("Regalo", callback_data='Ingreso_Regalo')],
            [InlineKeyboardButton("Otro", callback_data='Otro')]
        ]
    else:
        description_buttons = [
            [InlineKeyboardButton("Salud", callback_data='Salud')],
            [InlineKeyboardButton("Impuesto", callback_data='Impuesto')],
            [InlineKeyboardButton("Recital", callback_data='Recital')],
            [InlineKeyboardButton("Salida", callback_data='Salida')],
            [InlineKeyboardButton("Hogar", callback_data='Hogar')],
            [InlineKeyboardButton("Supermercado", callback_data='Supermercado')],
            [InlineKeyboardButton("Transporte", callback_data='Transporte')],
            [InlineKeyboardButton("Mascota", callback_data='Mascota')],
            [InlineKeyboardButton("Donación", callback_data='Donación')],
            [InlineKeyboardButton("Auto", callback_data='Auto')],
            [InlineKeyboardButton("Estudio", callback_data='Estudio')],
            [InlineKeyboardButton("Gimnasio", callback_data='Gimnasio')],
            [InlineKeyboardButton("Regalo", callback_data='Gasto_Regalo')],
            [InlineKeyboardButton("Banco/Resumen", callback_data='Banco/Resumen')],
            [InlineKeyboardButton("Viaje", callback_data='Viaje')],
            [InlineKeyboardButton("Otro", callback_data='Otro')]
        ]
    
    # Pregunto por Descripcion
    await update.message.reply_text(
        "Elegí la **descripción**:",
        parse_mode='Markdown',
        reply_markup=InlineKeyboardMarkup(description_buttons)
    )
    return DESCRIPTION

# Función para manejar la descripción
async def description(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Guardo Descripcion en la base
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['description'] = query.data
    logging.info("Descripción de %s: %s", user.first_name, query.data)
    
    payment_buttons = [
        [InlineKeyboardButton("Efectivo Pesos", callback_data='Efectivo Pesos')],
        [InlineKeyboardButton("Efectivo Dólares", callback_data='Efectivo Dólares')],
        [InlineKeyboardButton("Transferencia MP", callback_data='Transferencia MP')],
        [InlineKeyboardButton("Transferencia Naranja", callback_data='Transferencia Naranja')],
        [InlineKeyboardButton("Débito Naranja", callback_data='Débito Naranja')],
        [InlineKeyboardButton("Crédito Naranja", callback_data='Crédito Naranja')],
        [InlineKeyboardButton("Crédito Visa Naranja", callback_data='Crédito Visa Naranja')],
        [InlineKeyboardButton("Transferencia Galicia", callback_data='Transferencia Galicia')],
        [InlineKeyboardButton("Débito Galicia", callback_data='Débito Galicia')],
        [InlineKeyboardButton("Crédito Visa Galicia", callback_data='Crédito Visa Galicia')],
        [InlineKeyboardButton("Crédito Master Galicia", callback_data='Crédito Master Galicia')],
        [InlineKeyboardButton("Débito Macro", callback_data='Débito Macro')],
        [InlineKeyboardButton("Crédito Visa Macro", callback_data='Crédito Visa Macro')],
        [InlineKeyboardButton("Crédito Master Macro", callback_data='Crédito Master Macro')],
    ]
    
    await query.message.reply_text(
        "¿De qué **forma** se realizó el movimiento?",
        parse_mode='Markdown',
        reply_markup=InlineKeyboardMarkup(payment_buttons)
    )
    return PAYMENT_METHOD

# Comentario
async def payment_method(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    query = update.callback_query
    await query.answer()
    user = query.from_user
    context.user_data['payment_method'] = query.data
    logging.info("Forma de pago de %s: %s", user.first_name, query.data)

    await query.message.reply_text(
        "¿Queres agregar un **comentario**?",
        parse_mode='Markdown',
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return COMMENT

# Función para manejar el comentario o continuar si es 'No'
async def comment(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Verifico si el usuario quiere agregar un comentario
    if update.message.text == 'Sí':
        context.user_data['awaiting_comment'] = True  # Indicamos que estamos esperando un comentario
        await update.message.reply_text("Comentario:")
        return CONTINUE
    else:
        # Si el usuario no quiere agregar un comentario, guardo un valor vacío
        context.user_data['comment'] = ""  # Comentario vacío si el usuario dijo "No"

        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # No estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿Queres registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

# Función para manejar la respuesta del comentario
async def handle_comment_text(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Solo se ejecuta si estábamos esperando un comentario
    if context.user_data.get('awaiting_comment', False):
        # Guardo el comentario en la base de datos
        user = update.message.from_user
        context.user_data['comment'] = update.message.text
        logging.info("Comentario de %s: %s", user.first_name, update.message.text)
        
        # Guardar la información en un CSV
        guardar_datos(context.user_data)
        
        # Pregunto por otro movimiento
        context.user_data['awaiting_comment'] = False  # Ya no estamos esperando un comentario
        await update.message.reply_text(
            "Muchas gracias, ¿Queres registrar otro movimiento?",
            reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
        )
        return CONTINUE

    # Si no se esperaba un comentario, manejamos la respuesta de continuar
    return await continue_conversation(update, context)

# Función para manejar la cuenta elegida
async def update_account(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data['account'] = update.message.text
    await update.message.reply_text(
        f"Ingrese el nuevo saldo para la cuenta {update.message.text}:"
    )
    return UPDATE_AMOUNT

async def consult_account(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    account = update.message.text
    saldo = obtener_saldo(account)
    await update.message.reply_text(f"El saldo de la cuenta {account} es {saldo}.")
    await update.message.reply_text(
        "¿Quieres realizar otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

# Función para actualizar el saldo con el nuevo monto
async def update_amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    account = context.user_data['account']
    new_amount = update.message.text
    actualizar_saldo(account, new_amount)
    await update.message.reply_text(
        f"El saldo de {account} ha sido actualizado a {new_amount}."
    )
    await update.message.reply_text(
        "¿Quieres realizar otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

# Función para consultar el saldo de una cuenta
async def consultar_saldo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    user = update.message.from_user
    cuenta = update.message.text
    context.user_data['account'] = cuenta
    logging.info("Consulta de saldo por %s: %s", user.first_name, cuenta)
    
    # Obtener saldo de la cuenta
    saldo = obtener_saldo(cuenta)
    
    # Devolver el saldo al usuario
    await update.message.reply_text(f"El saldo de la cuenta {cuenta} es {saldo}.")
    
    # Preguntar si quiere hacer otra operación
    await update.message.reply_text(
        "¿Queres hacer otra operación?",
        reply_markup=ReplyKeyboardMarkup([['Sí', 'No']], one_time_keyboard=True)
    )
    return CONTINUE

# Función para obtener el saldo de la cuenta desde saldos.csv
def obtener_saldo(cuenta):
    try:
        df_saldos = pd.read_csv("saldos.csv")
        if cuenta in df_saldos["cuenta"].values:
            saldo = df_saldos.loc[df_saldos["cuenta"] == cuenta, "saldo"].values[0]
            return saldo
        else:
            return "Cuenta no encontrada"
    except FileNotFoundError:
        return "Archivo de saldos no encontrado"

# Función para manejar la continuación
async def continue_conversation(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Comienzo de nuevo si la respuesta es 'Sí'
    if update.message.text == 'Sí':
        return await start(update, context)
    
    # Termino la conversación si la respuesta es 'No'
    else:
        await update.message.reply_text("Gracias por usar TuCapybaraBot. ¡Hasta luego!")
        return ConversationHandler.END

# Función para guardar datos en un CSV
def guardar_datos(data):
    try:
        df = pd.read_csv("finanzas.csv")
    except FileNotFoundError:
        df = pd.DataFrame(columns=["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"])

    # Crear un DataFrame temporal con los nuevos datos
    new_data = pd.DataFrame([data])

    # Asignar número de movimiento (número de fila)
    new_data["numero_movimiento"] = len(df) + 1
    
    # Asignar fecha actual
    new_data["fecha"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    # Reordenar las columnas para tener "numero_movimiento" al principio y "fecha" al final
    new_data = new_data[["numero_movimiento", "movement_type", "amount", "currency", "description", "payment_method", "comment", "fecha"]]

    # Concatenar el DataFrame temporal con el DataFrame existente
    df = pd.concat([df, new_data], ignore_index=True)
    df.to_csv("finanzas.csv", index=False)

    # Actualizar el saldo en función del tipo de movimiento y el método de pago
    actualizar_saldo_por_movimiento(data)

# Función para actualizar el saldo basado en el movimiento registrado
def actualizar_saldo_por_movimiento(data):
    cuenta = None
    if data['payment_method'] == 'Efectivo Pesos':
        cuenta = 'Efectivo Pesos'
    elif data['payment_method'] == 'Efectivo Dólares':
        cuenta = 'Efectivo Dólares'
    elif data['payment_method'] == 'Transferencia MP':
        cuenta = 'Mercado Pago'
    elif data['payment_method'] in ['Transferencia Naranja', 'Débito Naranja']:
        cuenta = 'Naranja'
    elif data['payment_method'] in ['Transferencia Galicia', 'Débito Galicia']:
        cuenta = 'CA $ Galicia'

    if cuenta:
        try:
            # Convierte el monto a un número
            monto = float(data['amount'])
            # Si es un gasto, se convierte en negativo
            if data['movement_type'] == '🚨 Gasto':
                monto = -monto

            # Actualiza el saldo en el archivo saldos.csv
            actualizar_saldo(cuenta, monto, modificar=True)
        except ValueError:
            print("Error al convertir el monto del movimiento.")

# Modificar la función de actualizar saldo para sumar o restar
def actualizar_saldo(cuenta, monto, modificar=False):
    try:
        df_saldos = pd.read_csv("saldos.csv")
    except FileNotFoundError:
        df_saldos = pd.DataFrame(columns=["cuenta", "saldo", "fecha_actualizacion"])

    # Obtener la fecha actual
    fecha_actualizacion = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    # Actualizar el saldo de la cuenta especificada y agregar la fecha de actualización
    if cuenta in df_saldos["cuenta"].values:
        if modificar:
            # Si es un ajuste (suma o resta), se modifica el saldo actual
            saldo_actual = df_saldos.loc[df_saldos["cuenta"] == cuenta, "saldo"].values[0]
            nuevo_saldo = float(saldo_actual) + monto
            df_saldos.loc[df_saldos["cuenta"] == cuenta, ["saldo", "fecha_actualizacion"]] = [nuevo_saldo, fecha_actualizacion]
        else:
            # Si es una actualización directa de monto (ej. actualizar saldo manualmente)
            df_saldos.loc[df_saldos["cuenta"] == cuenta, ["saldo", "fecha_actualizacion"]] = [monto, fecha_actualizacion]
    else:
        # Si la cuenta no existe, la agrega
        new_row = pd.DataFrame([[cuenta, monto, fecha_actualizacion]], columns=["cuenta", "saldo", "fecha_actualizacion"])
        df_saldos = pd.concat([df_saldos, new_row], ignore_index=True)

    df_saldos.to_csv("saldos.csv", index=False)

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text('Operación cancelada. ¡Hasta luego!')
    return ConversationHandler.END

def main() -> None:
    application = Application.builder().token(TOKEN).build()
    
    conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        ACTION_TYPE: [MessageHandler(filters.Regex('^(➕ Agregar Movimiento|💰 Actualizar Saldos|🔍 Consultar Saldos)$'), action_type)],
        MOVEMENT_TYPE: [MessageHandler(filters.Regex('^(🤑 Ingreso|🚨 Gasto)$'), movement_type)],
        UPDATE_ACCOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_account)],
        CONSULT_ACCOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, consult_account)],
        UPDATE_AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, update_amount)],
        AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, amount)],
        CURRENCY: [MessageHandler(filters.TEXT & ~filters.COMMAND, currency)],
        COMMENT: [MessageHandler(filters.Regex('^(Sí|No)$'), comment)],
        CONTINUE: [MessageHandler(filters.Regex('^(Sí|No)$'), continue_conversation)],
        DESCRIPTION: [CallbackQueryHandler(description)],
        PAYMENT_METHOD: [CallbackQueryHandler(payment_method)],
        CONTINUE: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_comment_text)],
    },
    fallbacks=[CommandHandler('cancel', cancel)],
)

    application.add_handler(conv_handler)

    application.run_polling()

if __name__ == '__main__':
    main() 